In [1]:
from dataset_UNSW_NB15 import Dataset_UNSW_NB15
from dataset_NSL_KDD import Dataset_NSL_KDD
from dataset_IoT_Network_Intrusion_Dataset import Dataset_IoT_Network_Intrusion_Dataset
from experiment import Experiment


In [2]:
dataset1 = Dataset_NSL_KDD(config_filename="config_NSL_KDD.json")
exp1 = Experiment(dataset1.data, dataset1.labels, dataset1.config)
exp1.run(model_params={"k": 500 , "n_features" : 30})

#dataset2 = Dataset_UNSW_NB15(config_filename="config_UNSW_NB15.json")
#exp2 = Experiment(dataset2.data, dataset2.labels, dataset2.config)
#exp2.run(model_params={"k": 500 , "n_features" : 30})

#dataset3 = Dataset_IoT_Network_Intrusion_Dataset(config_filename="config_IoT_Network_Intrusion_Dataset.json")
#exp3 = Experiment(dataset3.data, dataset3.labels, dataset3.config)
#exp3.run(model_params={"k": 500 , "n_features" : 30})


/Users/ham/codes/IDS_IoT/dataset_UNSW_NB15.py:24: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(self.DATA_FILE_PATH, nrows=nrows)


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']